In [77]:
#import the dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [45]:
# loading and analysing the dataset
diab = pd.read_csv('diabetes.csv')

In [46]:
diab.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [47]:
diab.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [48]:
diab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [49]:
diab.shape

(768, 9)

In [50]:
diab['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [51]:
diab.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [52]:
#separating features and label
X = diab.drop(columns='Outcome', axis=1)
y = diab['Outcome']
print(X.shape, y.shape)

(768, 8) (768,)


In [53]:
smote = SMOTE(sampling_strategy='auto' , random_state=42)
X_resampled, y_resampled = smote.fit_resample(X,y)
print(pd.Series(y_resampled).value_counts())

Outcome
1    500
0    500
Name: count, dtype: int64


In [54]:
print(X_resampled.shape, y_resampled.shape)

(1000, 8) (1000,)


In [55]:
#train and split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled ,y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

In [69]:
#standardizing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [70]:
param_grid = {'C': [0.1, 1, 10, 100], 'gamma':['scale', 'auto', 0.01, 0.1, 1], 'kernel': ['rbf', 'linear']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


In [78]:
#Training the model
model = grid_search.best_estimator_
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print("Cross-validation accuracy scores:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))

Cross-validation accuracy scores: [0.8375  0.78125 0.775   0.84375 0.79375]
Mean accuracy: 0.80625


In [79]:
model.fit(X_train_scaled,y_train)

SVC(C=10, gamma='auto')

In [80]:
#evaluating the model using accuracy score
X_train_pred = model.predict(X_train_scaled)
X_train_acc = classification_report(X_train_pred, y_train)
print(f'Report of training data: {X_train_acc}')

Report of training data:               precision    recall  f1-score   support

           0       0.89      0.91      0.90       391
           1       0.92      0.89      0.90       409

    accuracy                           0.90       800
   macro avg       0.90      0.90      0.90       800
weighted avg       0.90      0.90      0.90       800



In [81]:
#test
X_test_pred = model.predict(X_test_scaled)
X_test_acc = classification_report(X_test_pred, y_test)
print(f'Report of test data: {X_test_acc}')

Report of test data:               precision    recall  f1-score   support

           0       0.81      0.83      0.82        98
           1       0.83      0.81      0.82       102

    accuracy                           0.82       200
   macro avg       0.82      0.82      0.82       200
weighted avg       0.82      0.82      0.82       200



In [82]:
def get_user_input():
    while True:
        try:
            user_input = input('Enter 8 values comma-separated for prediction: ')
            values = tuple(map(float, user_input.split(',')))
            if len(values) != 8:
                print("Error: Please enter exactly 8 float values")
                continue
            return values
        except ValueError:
            print("Error: All values must be numbers. please try again")

In [98]:
input_data = get_user_input()
#make prediction
input_data_array = np.asarray(input_data)
input_data_reshaped = input_data_array.reshape(1,-1)
std_data = scaler.transform(input_data_reshaped) #to standardize input data
print(std_data)
prediction = model.predict(std_data)
print(prediction)

Enter 8 values comma-separated for prediction:  2,106.2,77.0,12.5,135.54588,26.63,0.98,30


[[-0.58556339 -0.60971955  0.40725213 -0.51849361  0.44696723 -0.79991656
   1.44506886 -0.35413556]]
[1]


C:\Users\hp\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
